In [199]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [200]:
words = open('names.txt', 'r').read().splitlines()
len(words)

32033

In [201]:
chars = sorted(list(set(''.join(words))))
chars = ['.'] + chars
stoi = {n:i for i,n in enumerate(chars)}
itos = {n:i for i,n in stoi.items()}

In [202]:
block_size = 3
X, Y = [], []
for w in words:
    context = '.'*block_size
    for i, ch in enumerate(w + '.'):
        curr_x = context
        req = block_size - len(context)
        temp_w = w[:i]
        temp_w = temp_w[len(temp_w)-req:]
        curr_x += temp_w
        context = context[:-1]
#         print(curr_x, "---->", curr_y)
        X.append([stoi[x_ch] for x_ch in curr_x])
        Y.append(stoi[ch])
X = torch.tensor(X)
Y = torch.tensor(Y)
X.shape, Y.shape

(torch.Size([228146, 3]), torch.Size([228146]))

In [198]:
# C = torch.randn((len(chars), 2))

In [197]:
# emb = C[X]
# emb.shape

In [166]:
# W1 = torch.randn((6, 100), requires_grad=True)
# b1 = torch.randn(100)
# W1.shape, b1.shape

In [150]:
### Exploring pytorch ###

# torch.cat(torch.unbind(emb, 1), 1)
# torch.unbind(emb, 1)
# emb.storage()
# emb.view(12, 6)
    # emb.view(-1, 6) # -> here -1 take cares about the remaining count like 12*6/6

In [165]:
# A1 = torch.tanh(emb.view(-1, 6) @ W1 + b1)
# A1.shape

In [160]:
# W2 = torch.randn((100, len(chars)), requires_grad=True)
# b2 = torch.randn(len(chars))

In [164]:
######## my try ##########

# logits = A1 @ W2 + b2
# counts = logits.exp()
# probs = counts / counts.sum(1, keepdims=True)
# loss = -probs[torch.arange(12), Y].log().mean()
# print(loss)

# W2.grad = None
# W1.grad = None
# loss.backward(retain_graph=True)

# W2.data += -10*W2.grad
# W1.data += -10*W1.grad

###########################

In [167]:
######## Making respectable :) ########

In [203]:
X.shape, Y.shape

(torch.Size([228146, 3]), torch.Size([228146]))

In [204]:
# parameters
C = torch.randn((len(chars), 2))
W1 = torch.randn((6, 100))
b1 = torch.randn(100)
W2 = torch.randn((100, len(chars)))
b2 = torch.randn(len(chars))
parameters = [C, W1, b1, W2, b2]

In [205]:
# setting require_grad to True for parameters
for p in parameters:
    p.requires_grad = True
# parameters in total
sum(p.nelement() for p in parameters)

3481

In [207]:
# for _ in range(10):
#     ## forward
#     emb = C[X]
#     A1 = torch.tanh(emb.view(-1, 6) @ W1 + b1)
#     logits = A1 @ W2 + b2

#     # # now applying softmax
#     # counts = logits.exp()
#     # probs = counts / counts.sum(1, keepdims=True)
#     # # calculating loss by mean of negative log likelihoods
#     # loss = -probs[torch.arange(12), Y].log().mean()

#     # In above lines we are calculating softmax and then loss
#     # This is common and we do have a pytorch inbuilt function pytorch.nn.functional.cross_entropy()
#     # >> Which computes cross entropy between input and target (here logits and Y)...

#     loss = F.cross_entropy(logits, Y)

#     ## backward
#     for p in parameters:
#         p.grad = None
#     loss.backward()

#     ## updation
#     for p in parameters:
#         p.data += -0.1 * p.grad

#     print(loss.item())

In [212]:
for _ in range(10):
    ## training through batches - batch construct
    ix = torch.randint(0, X.shape[0], (32,))
    
    ## forward
    emb = C[X[ix]]
    A1 = torch.tanh(emb.view(-1, 6) @ W1 + b1)
    logits = A1 @ W2 + b2

    # # now applying softmax
    # counts = logits.exp()
    # probs = counts / counts.sum(1, keepdims=True)
    # # calculating loss by mean of negative log likelihoods
    # loss = -probs[torch.arange(12), Y].log().mean()

    # In above lines we are calculating softmax and then loss
    # This is common and we do have a pytorch inbuilt function pytorch.nn.functional.cross_entropy()
    # >> Which computes cross entropy between input and target (here logits and Y)...

    loss = F.cross_entropy(logits, Y[ix])

    ## backward
    for p in parameters:
        p.grad = None
    loss.backward()

    ## updation
    for p in parameters:
        p.data += -0.1 * p.grad

    print(loss.item())

torch.Size([32])
5.268311500549316
torch.Size([32])
7.266613006591797
torch.Size([32])
4.971206188201904
torch.Size([32])
7.393373489379883
torch.Size([32])
6.548801422119141
torch.Size([32])
5.401449680328369
torch.Size([32])
5.703242778778076
torch.Size([32])
5.228228569030762
torch.Size([32])
6.0923004150390625
torch.Size([32])
5.413201808929443


In [209]:
X.shape

torch.Size([228146, 3])